In [14]:
#importing library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier,XGBRegressor

In [15]:
#loading the dataset
df=pd.read_csv('/kaggle/input/smart-grid-stability/smart_grid_stability_augmented.csv')
#showing the dataset
df

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2.930406,2.376523,9.487627,6.187797,3.343416,-1.449106,-0.658054,-1.236256,0.601709,0.813512,0.779642,0.608385,0.023892,unstable
59996,3.392299,2.954947,1.274827,6.894759,4.349512,-0.952437,-1.663661,-1.733414,0.502079,0.285880,0.567242,0.366120,-0.025803,stable
59997,2.364034,8.776391,2.842030,1.008906,4.299976,-0.943884,-1.380719,-1.975373,0.487838,0.149286,0.986505,0.145984,-0.031810,stable
59998,9.631511,2.757071,3.994398,7.821347,2.514755,-0.649915,-0.966330,-0.898510,0.365246,0.889118,0.587558,0.818391,0.037789,unstable


In [16]:
#getting preliminary information 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    60000 non-null  float64
 1   tau2    60000 non-null  float64
 2   tau3    60000 non-null  float64
 3   tau4    60000 non-null  float64
 4   p1      60000 non-null  float64
 5   p2      60000 non-null  float64
 6   p3      60000 non-null  float64
 7   p4      60000 non-null  float64
 8   g1      60000 non-null  float64
 9   g2      60000 non-null  float64
 10  g3      60000 non-null  float64
 11  g4      60000 non-null  float64
 12  stab    60000 non-null  float64
 13  stabf   60000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 6.4+ MB


# #Preprocessing the Dataset

In [34]:
def preprocess_inputs(df,task='classification'):
    df=df.copy()
    if task=='classification':
        df=df.drop('stab',axis=1)
        y=df['stabf']
        x=df.drop('stabf',axis=1).copy()
    elif task=='regression':
        df=df.drop('stabf',axis=1)
        y=df['stab']
        x=df.drop('stab',axis=1).copy()
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,shuffle=True,random_state=123)
    return x_train,x_test,y_train,y_test
    

In [35]:
x_train,x_test,y_train,y_test=preprocess_inputs(df,task='classification')
x_train

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
50049,9.818128,8.588987,0.511597,5.260072,3.395636,-1.478677,-1.107857,-0.809102,0.931287,0.291643,0.599963,0.384295
42117,2.988689,0.532388,1.456258,9.804672,4.992851,-1.899322,-1.336375,-1.757154,0.945929,0.407395,0.236469,0.947458
20378,5.936029,9.336081,1.988640,1.907732,4.433782,-1.324150,-1.417504,-1.692128,0.452766,0.882208,0.159555,0.243988
17121,5.001385,3.720948,3.111653,8.465773,3.239639,-0.903370,-0.836304,-1.499965,0.129207,0.500248,0.863418,0.075551
22484,2.284157,4.881570,5.095673,8.647201,3.375693,-1.920184,-0.630081,-0.825428,0.477917,0.446620,0.773091,0.424167
...,...,...,...,...,...,...,...,...,...,...,...,...
54370,5.701267,6.684273,9.601239,1.441566,2.855926,-0.966439,-0.772487,-1.117001,0.767515,0.141450,0.825274,0.681713
17730,6.962724,5.720939,3.331487,3.904860,4.653462,-1.254036,-1.608298,-1.791128,0.207566,0.925921,0.999739,0.635747
28030,6.763179,4.443784,5.772680,8.344345,4.390494,-1.586663,-0.817729,-1.986103,0.449443,0.276032,0.990190,0.785749
15725,4.542570,5.302279,1.487013,6.115917,3.317675,-1.656348,-0.568968,-1.092359,0.279099,0.486512,0.411925,0.963123


In [36]:
y_train

50049    unstable
42117      stable
20378    unstable
17121      stable
22484    unstable
           ...   
54370    unstable
17730    unstable
28030    unstable
15725    unstable
52734    unstable
Name: stabf, Length: 42000, dtype: object

In [43]:
y_train=y_train.replace({'unstable':1,'stable':0})
y_test=y_test.replace({'unstable':1,'stable':0})

/tmp/ipykernel_30/1910803227.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test=y_test.replace({'unstable':1,'stable':0})


In [46]:
y_test

29561    1
26640    1
24498    1
24594    1
24249    1
        ..
59393    0
21497    0
18948    0
24384    1
48300    0
Name: stabf, Length: 18000, dtype: int64

In [44]:
clf=XGBClassifier()
clf.fit(x_train,y_train)
print('Classifier Trained')

Classifier Trained


In [48]:
print('The Classification Score',clf.score(x_test,y_test))

The Classification Score 0.9771111111111112


In [25]:
x_train,x_test,y_train,y_test=preprocess_inputs(df,task='regression')
x_train

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
50049,9.818128,8.588987,0.511597,5.260072,3.395636,-1.478677,-1.107857,-0.809102,0.931287,0.291643,0.599963,0.384295
42117,2.988689,0.532388,1.456258,9.804672,4.992851,-1.899322,-1.336375,-1.757154,0.945929,0.407395,0.236469,0.947458
20378,5.936029,9.336081,1.988640,1.907732,4.433782,-1.324150,-1.417504,-1.692128,0.452766,0.882208,0.159555,0.243988
17121,5.001385,3.720948,3.111653,8.465773,3.239639,-0.903370,-0.836304,-1.499965,0.129207,0.500248,0.863418,0.075551
22484,2.284157,4.881570,5.095673,8.647201,3.375693,-1.920184,-0.630081,-0.825428,0.477917,0.446620,0.773091,0.424167
...,...,...,...,...,...,...,...,...,...,...,...,...
54370,5.701267,6.684273,9.601239,1.441566,2.855926,-0.966439,-0.772487,-1.117001,0.767515,0.141450,0.825274,0.681713
17730,6.962724,5.720939,3.331487,3.904860,4.653462,-1.254036,-1.608298,-1.791128,0.207566,0.925921,0.999739,0.635747
28030,6.763179,4.443784,5.772680,8.344345,4.390494,-1.586663,-0.817729,-1.986103,0.449443,0.276032,0.990190,0.785749
15725,4.542570,5.302279,1.487013,6.115917,3.317675,-1.656348,-0.568968,-1.092359,0.279099,0.486512,0.411925,0.963123


In [49]:
y_train

50049    1
42117    0
20378    1
17121    0
22484    1
        ..
54370    1
17730    1
28030    1
15725    1
52734    1
Name: stabf, Length: 42000, dtype: int64

In [50]:
x_train,x_test,y_train,y_test=preprocess_inputs(df,task='regression')
clf=XGBRegressor()
clf.fit(x_train,y_train)
print('Classifier Trained')

Classifier Trained


In [51]:
print('Regression Score',clf.score(x_test,y_test))

Regression Score 0.9598745712139747
